In [1]:
import pandas as pd
import numpy as np

In [2]:
from metrics_f1 import calc_f1_score, calc_metrics

## Загрузка данных

In [3]:
path_train = r"./"

In [4]:
# данные по дислокации
dislok = pd.read_parquet(path_train + '/dislok_wagons.parquet').convert_dtypes()
# список вагонов с остаточным пробегом на момент прогноза
wag_prob = pd.read_parquet(path_train + '/wagons_probeg_ownersip.parquet').convert_dtypes()
# параметры вагона
wag_param = pd.read_parquet(path_train + '/wag_params.parquet').convert_dtypes()
# таргет по прогнозам выбытия вагонов в ПР на месяц и на 10 дней
target = pd.read_csv(path_train +'/target/y_train.csv').convert_dtypes()
# текущие ремонты вагонов
tr_rem = pd.read_parquet(path_train + '/tr_rems.parquet').convert_dtypes()

In [5]:
wag_param = wag_param.drop_duplicates(subset='wagnum', keep='last')# у вагонов могут меняться параметры, поэтмоу номер дублируется. В данной модели это фактор не учитывается

In [6]:
# !!!! month_to_predict = pd.to_datetime('2022-12-01')
month_to_predict = pd.to_datetime('2023-01-01')

In [7]:
target.month = pd.to_datetime(target.month)
target = target[target.month == month_to_predict][['wagnum','target_month','target_day']]

In [8]:
target.target_month.sum(), target.target_day.sum()

(1676, 461)

# Наивная модель

Наивная модель будет построена на правилах с использованием минимального набора данных, без применения Ml.

Реальный процесс выглядит следующим образом - в начале месяца берется срез по парку по всем вагонам, за ремонт которых несёт ответственность ПГК. Для выбранных вагонов требуется установить, какие из них будут отремонтированы в текущем месяце. Данная информация помогает планировать нагрузку на вагоно-ремонтное предприятие(ВРП). Вторая модель определяет критичные вагоны, которые будут отправлены в ремонт в первую очередь( в ближайшие 10 дней). Это помогает фокусировать внимание диспетчеров.

Основными критериями по которым вагон отправляется в плановый ремонт - является его остаточный пробег и срок до планового ремонта.
В регламентах РЖД используется следующее правило - если ресурс по пробегу не превышает 500 км и/или плановый ремонт должен наступить через 15 дней(или меньше), то вагон может ехать только на ВРП.
Из этого регламента вытекают две особенности:
1. Диспетчер старается отправить вагон раньше положенных значений. Это позволяет выбрать предприятия, на которых ремонтироваться дешевле, а не ближайшее.
2. Компания-оператор может выбирать какому из нормативов нужно следовать - ремонтировать вагон по сроку, или по пробегу, или по обоим критериям сразу. Поэтому встречаются вагоны, у которых пробег может не отслеживаться.

Вагон может быть отправлен в плановый ремонт и раньше положенного. На это может влиять, например, история грузовых операций и количество текущих(мелких) ремонтов. Основная цель участников в данной задаче - найти закономерности и оценить значимые признаки, по которым вагон выбывает в плановый ремонт.

## Подготовка данных

In [9]:
wag_prob[(wag_prob.repdate == pd.to_datetime('2023-01-26'))]

,repdate,wagnum,ost_prob,manage_type,rod_id,reestr_state,ownership_type,month
178,2023-01-26,33361,151775,0,1,1,0,1
451,2023-01-26,33364,151122,0,1,1,0,1
724,2023-01-26,33366,154991,0,1,1,0,1
997,2023-01-26,33358,25329,0,1,1,0,1
1270,2023-01-26,33349,145846,0,1,1,0,1
...,...,...,...,...,...,...,...,...
9248397,2023-01-26,17621,76882,0,1,1,0,1
9248670,2023-01-26,25045,140342,0,1,1,0,1
9248943,2023-01-26,27156,134112,0,1,1,0,1
9249216,2023-01-26,21361,130952,0,1,1,0,1


In [10]:
# оставим только данные по остаточному пробегу для каждого номерав вагона
wag_prob = wag_prob[(wag_prob.repdate == month_to_predict) | (wag_prob.repdate == wag_prob.repdate.min())]

In [11]:
wag_prob.head()

,repdate,wagnum,ost_prob,manage_type,rod_id,reestr_state,ownership_type,month
0,2022-08-01,33361,7541,0,1,1,0,8
153,2023-01-01,33361,153113,0,1,1,0,1
273,2022-08-01,33364,37103,0,1,1,0,8
426,2023-01-01,33364,157426,0,1,1,0,1
546,2022-08-01,33366,10242,0,1,1,0,8


In [12]:
# оценим среднесуточный пробег из данных по пробегу вагона, на тот случай, если данных по нормативу нет
wag_prob_ = wag_prob.groupby('wagnum', as_index = False).agg({'repdate':['max', 'min'] , 'ost_prob': ['max','min']},)#.droplevel(1)
wag_prob_.columns = [head+'_' + name
                     if head!='wagnum'
                     else head
                     for head, name in wag_prob_.columns ]

wag_prob_['diff_days'] = wag_prob_.repdate_max - wag_prob_.repdate_min
wag_prob_['mean_run'] = (wag_prob_.ost_prob_max - wag_prob_.ost_prob_min )/ wag_prob_.diff_days.dt.days

wag_prob_.head()

,wagnum,repdate_max,repdate_min,ost_prob_max,ost_prob_min,diff_days,mean_run
0,0,2023-01-01,2022-08-01,82413,57010,153 days 00:00:00,166.03268
1,1,2023-01-01,2022-08-01,132660,93069,153 days 00:00:00,258.764706
2,2,2023-01-01,2022-08-01,69345,56136,153 days 00:00:00,86.333333
3,3,2023-01-01,2022-08-01,83627,66998,153 days 00:00:00,108.686275
4,4,2023-01-01,2022-08-01,115869,97873,153 days 00:00:00,117.620915


In [13]:
wag_prob.head(10)

,repdate,wagnum,ost_prob,manage_type,rod_id,reestr_state,ownership_type,month
0,2022-08-01,33361,7541,0,1,1,0,8
153,2023-01-01,33361,153113,0,1,1,0,1
273,2022-08-01,33364,37103,0,1,1,0,8
426,2023-01-01,33364,157426,0,1,1,0,1
546,2022-08-01,33366,10242,0,1,1,0,8
699,2023-01-01,33366,159748,0,1,1,0,1
819,2022-08-01,33358,41827,0,1,1,0,8
972,2023-01-01,33358,28597,0,1,1,0,1
1092,2022-08-01,33349,5722,0,1,1,0,8
1245,2023-01-01,33349,150043,0,1,1,0,1


In [14]:
wag_prob = wag_prob[wag_prob.repdate == wag_prob.repdate.max()][['wagnum','ost_prob']]
wag_prob = wag_prob.merge(wag_prob_[['wagnum','mean_run']])

wag_prob.head()

,wagnum,ost_prob,mean_run
0,33361,153113,951.45098
1,33364,157426,786.424837
2,33366,159748,977.163399
3,33358,28597,86.470588
4,33349,150043,943.27451


In [15]:
# для каждого вагона оставим только информацию по сроку службы и нормативу суточного пробега между ПР
wag_param = wag_param[['wagnum','srok_sl','cnsi_probeg_dr','cnsi_probeg_kr']]

wag_param.head()

,wagnum,srok_sl,cnsi_probeg_dr,cnsi_probeg_kr
3218,26318,2022-04-27,160,160
19128,28344,2024-12-24,110,160
21526,8099,2027-10-01,110,160
32353,33350,2047-02-05,250,500
81,5308,2027-09-28,110,160


In [16]:
# посчитаем сколько текущих ремонтов было за прошедший период
tr_rem = tr_rem.groupby('wagnum', as_index= False).kod_vrab.count()

tr_rem.head()

,wagnum,kod_vrab
0,0,2
1,2,2
2,3,2
3,6,9
4,8,2


In [17]:
# сохраним только дату следующего планового ремонта для вагона
dislok = dislok[['wagnum','date_pl_rem']].drop_duplicates(subset = 'wagnum', keep='last')

dislok.head()

,wagnum,date_pl_rem
347,11219,2019-06-27
25426,33350,2022-07-25
216913,8099,2023-04-20
730409,28344,2022-11-30
993683,26318,2023-01-01


In [18]:
# соберем все данные вместе
wp = target[['wagnum']].merge(wag_param, on ='wagnum', how = 'left')\
             .merge(wag_prob, how = 'left')\
             .merge(tr_rem, how = 'left')\
             .merge(dislok, how = 'left')

wp.head()

,wagnum,srok_sl,cnsi_probeg_dr,cnsi_probeg_kr,ost_prob,mean_run,kod_vrab,date_pl_rem
0,33361,2033-03-01,110,160,153113,951.45098,3.0,2023-02-17
1,33364,2031-04-12,110,160,157426,786.424837,2.0,2023-10-03
2,33366,2032-01-21,110,160,159748,977.163399,2.0,2023-04-03
3,33358,2032-11-30,110,160,28597,86.470588,2.0,2024-02-23
4,33349,2033-12-04,110,160,150043,943.27451,NaN,2023-07-06


In [19]:
# Получим среднесуточный пробег, как среднее от нормативов и реального пробега
wp[['cnsi_probeg_dr','cnsi_probeg_kr','mean_run']] = wp[['cnsi_probeg_dr','cnsi_probeg_kr','mean_run']].fillna(0)
wp['day_run'] = wp.apply(lambda x : [val  for val in [x.cnsi_probeg_kr, x.cnsi_probeg_dr, x.mean_run] if val != 0], axis = 1 )
wp['day_run']= wp.apply(lambda x : np.mean(x.day_run) if len(x.day_run)> 0 else 0, axis = 1 )
wp.head()

,wagnum,srok_sl,cnsi_probeg_dr,cnsi_probeg_kr,ost_prob,mean_run,kod_vrab,date_pl_rem,day_run
0,33361,2033-03-01,110,160,153113,951.45098,3.0,2023-02-17,407.150327
1,33364,2031-04-12,110,160,157426,786.424837,2.0,2023-10-03,352.141612
2,33366,2032-01-21,110,160,159748,977.163399,2.0,2023-04-03,415.721133
3,33358,2032-11-30,110,160,28597,86.470588,2.0,2024-02-23,118.823529
4,33349,2033-12-04,110,160,150043,943.27451,NaN,2023-07-06,404.424837


In [20]:
# !!!!
# x.cnsi_probeg_kr, x.cnsi_probeg_dr, x.mean_run - разных порядков, первые два в тыс. км, mean_run просто в км
# можно попробовать в day_run сразу записать mean_run, но
# ухудшает на сотую
'''
wp[['cnsi_probeg_dr','cnsi_probeg_kr','mean_run']] = wp[['cnsi_probeg_dr','cnsi_probeg_kr','mean_run']].fillna(0)
wp['day_run'] = wp['mean_run']
wp.head()
'''

"\nwp[['cnsi_probeg_dr','cnsi_probeg_kr','mean_run']] = wp[['cnsi_probeg_dr','cnsi_probeg_kr','mean_run']].fillna(0)\nwp['day_run'] = wp['mean_run']\nwp.head()\n"

In [21]:
wp['current_date'] = month_to_predict

In [22]:
# определим, сколько дней осталось до истечения срока службы
wp['date_diff_srk_sl'] = wp['srok_sl'] - wp['current_date']

In [23]:
# !!!!!!!
# вагоны, срок службы которых заканчивается точно не поедут на ПР. Исключим их в дальейшем
(wp['date_diff_srk_sl'] < pd.to_timedelta(0)).value_counts()

date_diff_srk_sl
False    33938
True        35
Name: count, dtype: int64

In [24]:
# определим, сколько дней осталось до ближайшего ПР
wp['date_diff_pl_rem'] = wp['date_pl_rem'] - wp['current_date']
wp.head(10)

,wagnum,srok_sl,cnsi_probeg_dr,cnsi_probeg_kr,ost_prob,mean_run,kod_vrab,date_pl_rem,day_run,current_date,date_diff_srk_sl,date_diff_pl_rem
0,33361,2033-03-01,110,160,153113,951.45098,3.0,2023-02-17,407.150327,2023-01-01,3712 days,47 days
1,33364,2031-04-12,110,160,157426,786.424837,2.0,2023-10-03,352.141612,2023-01-01,3023 days,275 days
2,33366,2032-01-21,110,160,159748,977.163399,2.0,2023-04-03,415.721133,2023-01-01,3307 days,92 days
3,33358,2032-11-30,110,160,28597,86.470588,2.0,2024-02-23,118.823529,2023-01-01,3621 days,418 days
4,33349,2033-12-04,110,160,150043,943.27451,NaN,2023-07-06,404.424837,2023-01-01,3990 days,186 days
5,33354,2031-03-24,110,160,15237,100.823529,NaN,2023-07-29,123.607843,2023-01-01,3004 days,209 days
6,33355,2031-12-17,110,160,8730,121.071895,2.0,2024-01-14,130.357298,2023-01-01,3272 days,378 days
7,33356,2033-11-17,110,160,141461,860.535948,NaN,2022-09-28,376.845316,2023-01-01,3973 days,-95 days
8,33370,2032-08-28,110,160,152431,880.026144,NaN,2023-07-04,383.342048,2023-01-01,3527 days,184 days
9,33373,2050-02-12,350,500,343038,2135.098039,NaN,2023-03-20,995.032680,2023-01-01,9904 days,78 days


In [25]:
# !!!!!!!
# вагоны, которые уйдут на ремонт до нашей даты. Исключим их в дальейшем
(wp['date_diff_pl_rem'] < pd.to_timedelta(0)).value_counts()

date_diff_pl_rem
False    31025
True      2948
Name: count, dtype: int64

In [26]:
# !!!!! 
# заполним пропуски для своих расчётов здесь, чтобы prob_end_month тоже был без пропусков
wp.ost_prob = wp.ost_prob.fillna(160000)
wp.day_run = wp.day_run.fillna(wp.day_run.mean())

In [27]:
wp

,wagnum,srok_sl,cnsi_probeg_dr,cnsi_probeg_kr,ost_prob,mean_run,kod_vrab,date_pl_rem,day_run,current_date,date_diff_srk_sl,date_diff_pl_rem
0,33361,2033-03-01,110,160,153113,951.45098,3.0,2023-02-17,407.150327,2023-01-01,3712 days,47 days
1,33364,2031-04-12,110,160,157426,786.424837,2.0,2023-10-03,352.141612,2023-01-01,3023 days,275 days
2,33366,2032-01-21,110,160,159748,977.163399,2.0,2023-04-03,415.721133,2023-01-01,3307 days,92 days
3,33358,2032-11-30,110,160,28597,86.470588,2.0,2024-02-23,118.823529,2023-01-01,3621 days,418 days
4,33349,2033-12-04,110,160,150043,943.27451,NaN,2023-07-06,404.424837,2023-01-01,3990 days,186 days
...,...,...,...,...,...,...,...,...,...,...,...,...
33968,17621,2032-11-12,110,160,83554,187.679739,NaN,2024-11-09,152.559913,2023-01-01,3603 days,678 days
33969,25045,2034-08-30,110,160,143729,820.575163,NaN,2023-06-29,363.525054,2023-01-01,4259 days,179 days
33970,27156,2030-12-18,110,160,146545,886.052288,4.0,2023-06-27,385.350763,2023-01-01,2908 days,177 days
33971,21361,2028-03-04,160,160,136814,795.54902,NaN,2023-06-08,371.849673,2023-01-01,1889 days,158 days


In [28]:
# определим, какой остаточный ресурс будет на момент окончания месяца
wp['prob_end_month'] = wp['ost_prob'] - wp['day_run'] * 30

In [29]:
wp.head(10)

,wagnum,srok_sl,cnsi_probeg_dr,cnsi_probeg_kr,ost_prob,mean_run,kod_vrab,date_pl_rem,day_run,current_date,date_diff_srk_sl,date_diff_pl_rem,prob_end_month
0,33361,2033-03-01,110,160,153113,951.45098,3.0,2023-02-17,407.150327,2023-01-01,3712 days,47 days,140898.490196
1,33364,2031-04-12,110,160,157426,786.424837,2.0,2023-10-03,352.141612,2023-01-01,3023 days,275 days,146861.751634
2,33366,2032-01-21,110,160,159748,977.163399,2.0,2023-04-03,415.721133,2023-01-01,3307 days,92 days,147276.366013
3,33358,2032-11-30,110,160,28597,86.470588,2.0,2024-02-23,118.823529,2023-01-01,3621 days,418 days,25032.294118
4,33349,2033-12-04,110,160,150043,943.27451,NaN,2023-07-06,404.424837,2023-01-01,3990 days,186 days,137910.254902
5,33354,2031-03-24,110,160,15237,100.823529,NaN,2023-07-29,123.607843,2023-01-01,3004 days,209 days,11528.764706
6,33355,2031-12-17,110,160,8730,121.071895,2.0,2024-01-14,130.357298,2023-01-01,3272 days,378 days,4819.281046
7,33356,2033-11-17,110,160,141461,860.535948,NaN,2022-09-28,376.845316,2023-01-01,3973 days,-95 days,130155.640523
8,33370,2032-08-28,110,160,152431,880.026144,NaN,2023-07-04,383.342048,2023-01-01,3527 days,184 days,140930.738562
9,33373,2050-02-12,350,500,343038,2135.098039,NaN,2023-03-20,995.032680,2023-01-01,9904 days,78 days,313187.019608


In [30]:
# !!!!!!!!!!!!
# у нас есть prob_end_month которые отрицательные, то есть их ресурс закончится до конца месяца.
# Но у некоторых ресурс закончится сильно заранее, то есть ещё до начала месяца.
# Нужно вычислить эти случаи, разделив отрицательные prob_end_month на среднесуточный пробег,
# получив тем самым количество дней, насколько раньше конца месяца закончится ресурс,
# и откинув те вагоны, ресурс которых закончится раньше, чем за 30 дней до конца месяца (это потом).
wp['prob_end_month_fixed'] = wp.apply(lambda x: x.prob_end_month / x.day_run if x.prob_end_month < 0 else x.prob_end_month, axis=1)

In [31]:
# !!!!!
# prob_end_month_fixed у 14003 -63.9 - то есть ресурс закончится за 64 дня до конца месяца. Исключим его потом.
wp[(wp.prob_end_month_fixed < -31)].head()

,wagnum,srok_sl,cnsi_probeg_dr,cnsi_probeg_kr,ost_prob,mean_run,kod_vrab,date_pl_rem,day_run,current_date,date_diff_srk_sl,date_diff_pl_rem,prob_end_month,prob_end_month_fixed
1002,11772,2024-12-30,110,160,-859,240.633987,3.0,2023-12-14,170.211329,2023-01-01,729 days,347 days,-5965.339869,-35.046668
2342,12577,2033-01-25,110,160,-1332,178.228758,NaN,2023-09-29,149.409586,2023-01-01,3677 days,271 days,-5814.287582,-38.915091
4685,21098,2032-02-21,110,160,-277,192.261438,NaN,2024-04-28,154.087146,2023-01-01,3338 days,483 days,-4899.614379,-31.797684
4843,21607,2032-04-16,110,160,-164,207.810458,4.0,2023-08-31,159.270153,2023-01-01,3393 days,242 days,-4942.104575,-31.029697
4933,21176,2030-02-20,110,160,-181,160.594771,NaN,2024-02-20,143.531590,2023-01-01,2607 days,415 days,-4486.947712,-31.261046


In [32]:
# !!!!!!!!!
wp[(wp.prob_end_month_fixed < -31)]['wagnum'].count()

75

In [33]:
wp['target_month'] = 0

# вагон выбывает в ПР в следующем месяце, если:
# остаточный пробег < 5 000 км
# срок службы < 500 дней
# до следующего  ПР < 40 дней
# ,число текущих ремонтов > 5
wp.loc[(wp.prob_end_month <= 5000) & (wp.prob_end_month_fixed > -30)\
       | (wp.date_diff_srk_sl < pd.to_timedelta(500)) & (wp.date_diff_srk_sl > pd.to_timedelta(0))\
        | (wp.date_diff_pl_rem < pd.to_timedelta(40)) & (wp.date_diff_pl_rem > pd.to_timedelta(0)) \
        | (wp.kod_vrab > 50),'target_month'] = 1


# замена wp.kod_vrab > 5 на 50 поднимает метрику до 0.271, то есть стоит отказаться от этого параметра
# (wp.prob_end_month_fixed > -30) немного ухудшает, на сотую
# wp.date_diff_srk_sl > pd.to_timedelta(0) практически не влияет
# (wp.date_diff_pl_rem > pd.to_timedelta(0)) поднимает до 0.430

In [34]:
(wp['target_month'] == 1).value_counts()

target_month
False    31877
True      2096
Name: count, dtype: int64

In [35]:
# ((wp.prob_end_month <= 5000) & (wp.prob_end_month > 0)).value_counts()

# даёт 0.17 вместо 0.43

In [36]:
(wp.date_diff_srk_sl < pd.to_timedelta(500)).value_counts()

date_diff_srk_sl
False    33938
True        35
Name: count, dtype: int64

In [37]:
(wp.date_diff_pl_rem < pd.to_timedelta(10)).value_counts()

date_diff_pl_rem
False    31023
True      2950
Name: count, dtype: int64

In [38]:
((wp.date_diff_pl_rem < pd.to_timedelta(40)) & (wp.date_diff_pl_rem > pd.to_timedelta(0))).value_counts()

date_diff_pl_rem
False    33973
Name: count, dtype: int64

In [39]:
(wp.kod_vrab > 10).value_counts()

kod_vrab
False    33949
True        24
Name: count, dtype: int64

In [40]:
wp['target_day'] = wp['target_month']

In [41]:
pred_target = target[['wagnum']].merge(wp[['wagnum','target_month','target_day']],how = 'left')
pred_target = pred_target.drop_duplicates(subset = 'wagnum')

pred_target.head()

,wagnum,target_month,target_day
0,33361,0,0
1,33364,0,0
2,33366,0,0
3,33358,0,0
4,33349,0,0


In [42]:
# Проверим соотношение отмеченных вагонов с фактическим значением
round(pred_target.target_month.sum() / target.target_month.sum(), 2)

1.25

In [43]:
# сохраним таргет за месяц для выбранного периода отдельно
target_path = './prediction/target_predicton.csv'

pred_target.drop_duplicates(subset = 'wagnum').to_csv(target_path, index=False)

true_target_path = './prediction/target_predicton_true.csv'

target.drop_duplicates(subset = 'wagnum').to_csv(true_target_path, index=False)

In [44]:
all_targets = pd.merge(target, pred_target, on='wagnum')
all_targets.loc[(all_targets == 1).any(axis=1)].sort_values(by='wagnum').head(10)

,wagnum,target_month_x,target_day_x,target_month_y,target_day_y
30628,1,0,0,0,0
26132,247,1,0,0,0
26357,291,1,0,0,0
26694,297,1,0,1,1
26697,304,1,0,0,0
26702,323,1,0,0,0
15403,844,1,1,0,0
12389,849,1,0,0,0
28335,855,1,0,0,0
14633,861,1,1,0,0


In [45]:
target.target_month.value_counts()

target_month
0    32297
1     1676
Name: count, dtype: Int64

In [46]:
# оценим насколько хорошо удалось предсказать выбытие вагонов по месяцу и по 10 дням
calc_f1_score(true_target_path, target_path)

0.4332003387453235

Базовая метрика улучшенного бэйзлана - **0.433**

Базовая метрика бэйзлана - **0.23121**

In [47]:
print(calc_metrics(true_target_path, target_path))

(0.4332003387453235, 0.3511450381679389, 0.70725865737553)
